# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

I will try to use the sentiment analysis example introduced in the course but with the gpt-2

* PEFT technique: 
    - LoRA (Low Rank Adaptation) technique will be my choice 
* Model: 
    - GPT-2 is the chosen model as recommended.
* Evaluation approach: 
    - The `evaluate` and `Trainer` methods will be used for evaluation 
* Fine-tuning dataset: 
    - 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# from .autonotebook import tqdm as notebook_tqdm
# !pip install -q "datasets==2.15.0"

In [2]:
# !pip install -q "scikit-learn"

In [3]:
# from .autonotebook import tqdm as notebook_tqdm
from transformers import AutoModelForSequenceClassification, Trainer
from datasets import load_dataset, load_metric


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load the train and test split from the imdb dataset 
splits = ["train","test"]
data = load_dataset("imdb", split=splits)
data

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 122182.53 examples/s]


[Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 })]

In [33]:
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))
    
ds

# simpler way with from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(data["train"], test_size=0.2)

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [34]:
print(ds['train']['text'][0])
print(ds['train']['label'][0])

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
1


### Preproceess datasets

In [117]:
from transformers import AutoTokenizer, GPT2Tokenizer

# Specify model and tokenizer 
# Notice that gpt2 is well suited for language generation tasks. For sentiment analysis, using a 
# a model similar to the one used in the course (e.g distilbert-base-uncased) is more suitable.
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# "Asking to pad but the tokenizer does not have a padding token. "
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = {}

for split in splits:
    tokenized_ds[split] = ds[split].map(tokenize_function, batched=True)

# train_data = train_data.map(tokenize_function, batched=True)
# test_data = test_data.map(tokenize_function, batched=True)


In [110]:
from transformers import GPT2Tokenizer
from datasets import load_dataset

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)  # Update: set padding=True

# Add a new padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Assuming you have a list `splits` containing the names of your dataset splits, e.g., ['train', 'test']
tokenized_ds = {}

for split in splits:
    tokenized_ds[split] = ds[split].map(tokenize_function, batched=True)


Map: 100%|██████████| 500/500 [00:01<00:00, 299.67 examples/s]


In [36]:
# print(tokenized_ds['test'][0]['input_ids'])
print(tokenized_ds['test'][0]['input_ids'][:6])

[27, 1671, 1220, 6927, 1671, 11037]


In [85]:
tokenized_ds['test']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

### Foundation Model Training
- Freeze all parameters before training 
- Train the last layer 
- Metric function 

In [53]:
# Load and setup the model 
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                          num_labels=2,
                                                          id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                          label2id={"NEGATIVE": 0, "POSITIVE": 1})


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [55]:
# Set the model's pad token to match that of the tokenizer.
model.config.pad_token_id = tokenizer.pad_token_id

In [56]:
# Last leayer model classifier 
model.score

Linear(in_features=768, out_features=2, bias=False)

In [57]:
# compute metrics for Foundation model 
import numpy as np 
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Freeze the base model before training 
for param in model.base_model.parameters():
    param.requires_grad = False

In [58]:
# Create a Trainer and metric
from transformers import TrainingArguments
# metric = load_metric("accuracy")

training_args = TrainingArguments( 
            output_dir="./data/sentiment_analysis",
            learning_rate=2e-5,
            weight_decay=0.01,
            num_train_epochs= 1, 
            per_device_train_batch_size= 16,
            per_device_eval_batch_size = 16,
            evaluation_strategy= "epoch",
            save_strategy="epoch",
            load_best_model_at_end=True)

trainer = Trainer(model=model, 
                args=training_args,
                train_dataset=tokenized_ds['train'],
                eval_dataset=tokenized_ds['test'],
                tokenizer=tokenizer,
                data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                compute_metrics=compute_metrics
                 )

In [59]:
# train the Foundation model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.761451,0.510000


TrainOutput(global_step=32, training_loss=0.795929491519928, metrics={'train_runtime': 96.6459, 'train_samples_per_second': 5.174, 'train_steps_per_second': 0.331, 'total_flos': 261296750592000.0, 'train_loss': 0.795929491519928, 'epoch': 1.0})

In [60]:
# evaluate the performance of the trained model 
trainer.evaluate()

{'eval_loss': 0.7614508271217346,
 'eval_accuracy': 0.51,
 'eval_runtime': 46.1534,
 'eval_samples_per_second': 10.833,
 'eval_steps_per_second': 0.693,
 'epoch': 1.0}

In [61]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [ ]:
# The eval accuracy is actually not significanlty as good as a random guess 0.5
# Lets see if we can do better with peft later

In [48]:
# Look at some results 
import pandas as pd 

df = pd.DataFrame(tokenized_ds["test"])
df = df[["text","label"]]
df['text'] = df["text"].str.replace("<br />", " ")
predictions = trainer.predict(tokenized_ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)
df.head(10)

,text,label,predicted_label
0,When I unsuspectedly rented A Thousand Acres...,1,1
1,This is the latest entry in the long series of...,1,1
2,This movie was so frustrating. Everything seem...,0,1
3,"I was truly and wonderfully surprised at ""O' B...",1,1
4,This movie spends most of its time preaching t...,0,1
5,After a very long time Marathi cinema has come...,1,1
6,"This is a really sad, and touching movie! It d...",1,1
7,Don't pay any attention to the rave reviews of...,0,1
8,Porn legend Gregory Dark directs this cheesy h...,0,1
9,This was a great movie. Something not only for...,1,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [100]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     lora_dropout=0.1,
#     target_modules=['c_attn','c_proj'],
#     bias="none",
    task_type="TaskType.SEQ_CLS",
)
# LoraConfig(
# r=8, # Rank
# lora_alpha=32,
# target_modules=['c_attn', 'c_proj'],
# lora_dropout=0.1,
# bias="none",
# task_type=TaskType.SEQ_CLS
# )

In [118]:
from transformers import GPT2ForSequenceClassification
model_peft = GPT2ForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)
model_orig = model

# Set the model's pad token id to match the tokenizer's pad token id
model_peft.config.pad_token_id = tokenizer.pad_token_id

lora_model = get_peft_model(model_peft, lora_config)
print(lora_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModel(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
        

In [119]:
# check trainable parameter 
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.236428452686603


## Train the model 


In [120]:
train_args = TrainingArguments( 
            output_dir="./data/sentiment_analysis_lora",
            learning_rate=2e-5,
            num_train_epochs= 1, 
            per_device_train_batch_size= 32,
            per_device_eval_batch_size = 32,
            weight_decay = 0.01,
            evaluation_strategy= "epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            logging_dir='./logs')

trainer_lora = Trainer(
        model=lora_model,
        args=train_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics
    )

In [121]:
# train the model 
trainer_lora.train()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [86]:
tokenized_ds['test']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [87]:
tokenized_ds['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.